# Project 2: Exact, Transform, Load
## Housing market fluctations in four major cities of Texas - Rental Prices

### Group 2 Team Members:
* Shwet 'Sunny' Bhatt - ETL Houston Housing Price information
* Waynette Burke - ETL San Antonio and Houston Rental Information
* Ariana Garcia - ETL San Antonio Housing Price Information

In [1]:
#Import Dependency
import pandas as pd
import pymongo

In [2]:
# Automates browser actions
from splinter import Browser

# Parses the HTML
from unicodedata import normalize

# For scraping with Chrome
from webdriver_manager.chrome import ChromeDriverManager

### Establish a connection

In [3]:
# Initialize PyMongo to work with MongoDBs
client = pymongo.MongoClient('mongodb://localhost:27017')

In [4]:
# Define database
db = client.HousingMarket_db

#Creating collection("table") rentedInfo
coll_Rented = db.rentedInfo


#Creating collection("table") OwnedInfo
coll_Owned = db.OwnedInfo


In [5]:
#Dropping Database and Collection if exist
coll_Rented.drop()
coll_Owned.drop()

### Reading the CSV File for extraction

In [6]:
#CSV File to be extracted
houseRent = "Resources/Metro_zori_sm_month.csv"

#Read in CSV file
rental = pd.read_csv(houseRent)

In [7]:
#View of the imported CSV that was converted to a dataframe
rental.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,...,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31
0,102001,0,United States,country,NaN,1379.359350,1391.073873,1401.530422,1412.396507,1418.136828,...,1926.501816,1940.496249,1948.801048,1966.495998,1982.244729,2008.780414,2030.844688,2055.759958,2076.446338,2089.775788
1,394913,1,"New York, NY",msa,NY,2519.219640,2546.687664,2566.288864,2583.939076,2586.989228,...,2943.972800,2967.725193,2995.408435,3043.726843,3096.093502,3155.667377,3214.734293,3272.366446,3322.750066,3341.555894
2,753899,2,"Los Angeles, CA",msa,CA,1984.656244,1997.410467,2011.539491,2024.112854,2038.440690,...,2800.991074,2818.584204,2835.348908,2859.616662,2891.533834,2929.352690,2963.407518,2989.180066,3009.899282,3023.599107
3,394463,3,"Chicago, IL",msa,IL,1508.976397,1522.121857,1534.084115,1561.018730,1568.264918,...,1826.037645,1833.685056,1846.498235,1862.123404,1881.300441,1904.071425,1929.775478,1951.121693,1970.803363,1979.473043
4,394514,4,"Dallas, TX",msa,TX,1183.209930,1193.705541,1205.093370,1212.123457,1218.724732,...,1720.058793,1730.702831,1741.993929,1752.658761,1762.695569,1784.917301,1811.899478,1846.581556,1870.973903,1881.637168


In [8]:
#Snapshot of the column names to allow for selecting what is required for this analysis
rental.columns

Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30', '2015-07-31',
       '2015-08-31', '2015-09-30', '2015-10-31', '2015-11-30', '2015-12-31',
       '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30', '2016-05-31',
       '2016-06-30', '2016-07-31', '2016-08-31', '2016-09-30', '2016-10-31',
       '2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28', '2017-03-31',
       '2017-04-30', '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31',
       '2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31', '2018-01-31',
       '2018-02-28', '2018-03-31', '2018-04-30', '2018-05-31', '2018-06-30',
       '2018-07-31', '2018-08-31', '2018-09-30', '2018-10-31', '2018-11-30',
       '2018-12-31', '2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30',
       '2019-05-31', '2019-06-30', '2019-07-31', '2019-08-31', '2019-09-30',
       '2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29',
    

In [9]:
#Collecting data for the four Texas Cities: Dallas, Austin, San Antonio and Houston
texas_rental = rental.loc[((rental["RegionName"] == "Houston, TX") |
                                 (rental["RegionName"] == "San Antonio, TX")),:]
texas_rental.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,...,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31
5,394692,5,"Houston, TX",msa,TX,1266.485274,1276.757481,1285.512422,1294.301128,1298.504212,...,1545.519707,1556.718879,1556.678364,1564.092063,1569.778502,1584.517983,1598.195718,1612.944456,1625.251711,1632.930432
24,395055,24,"San Antonio, TX",msa,TX,1036.962690,1045.293207,1054.488948,1061.745970,1065.038288,...,1432.520254,1438.136386,1442.200762,1452.625453,1456.879987,1472.082994,1487.993268,1505.683417,1521.742595,1527.103719


In [10]:
#Filtering of the columns: Needing only data for the year 2021
reduced_rental = texas_rental[['RegionName','2021-01-31', '2021-02-28', '2021-03-31', '2021-04-30', '2021-05-31',
       '2021-06-30', '2021-07-31', '2021-08-31', '2021-09-30', '2021-10-31',
       '2021-11-30', '2021-12-31']]

reduced_rental.head()

,RegionName,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31
5,"Houston, TX",1385.370824,1381.69660,1387.381925,1400.492020,1427.686387,1459.167326,1491.475150,1518.166502,1530.272085,1540.927833,1545.519707,1556.718879
24,"San Antonio, TX",1244.323792,1246.63651,1256.555388,1272.033387,1293.879012,1319.928451,1350.828315,1387.116050,1415.238489,1430.159529,1432.520254,1438.136386


In [11]:
#Transpose the table data
transposed_house = reduced_rental.transpose()

#Reset the index so that the "Date" field is no longer considered as an Index
transposed_house = transposed_house.reset_index()

#renamed the columns
transposed_house = transposed_house.set_axis(["Date", "Houston,TX", "San Antonio, TX"], axis = "columns") 

# remove the first row which was the old header
transposed_house = transposed_house.drop([0])

#View of the modified dataframe
transposed_house.head()

,Date,"Houston,TX","San Antonio, TX"
1,2021-01-31,1385.370824,1244.323792
2,2021-02-28,1381.6966,1246.63651
3,2021-03-31,1387.381925,1256.555388
4,2021-04-30,1400.49202,1272.033387
5,2021-05-31,1427.686387,1293.879012


In [12]:
#Cleaning up the formatting of the data in the dataframe
transposed_house["Date"] =transposed_house["Date"].astype("datetime64")
transposed_house["Houston,TX"] = transposed_house["Houston,TX"].astype(float).map("${:,.2f}".format)
transposed_house["San Antonio, TX"] = transposed_house["San Antonio, TX"].astype(float).map("${:,.2f}".format)

transposed_house.head()

,Date,"Houston,TX","San Antonio, TX"
1,2021-01-31,"$1,385.37","$1,244.32"
2,2021-02-28,"$1,381.70","$1,246.64"
3,2021-03-31,"$1,387.38","$1,256.56"
4,2021-04-30,"$1,400.49","$1,272.03"
5,2021-05-31,"$1,427.69","$1,293.88"


### Adding dataframe to MongoDB

In [13]:
#Updating the MondoDB Collection with the data in the dataframe

rental_dict ={}

for row in transposed_house.index:
    #Creating the dictionary for the items in the dataframe
    rental_dict= {"date":transposed_house["Date"][row],
                  "Houston":transposed_house["Houston,TX"][row],
                  "San Antonio":transposed_house["San Antonio, TX"][row]}
    
    #Adding the dictionary to the database collection
    coll_Rented.insert_one(rental_dict)


In [14]:
# Display items in MongoDB collection
rentalTable = db.rentedInfo.find()

for entry in rentalTable:
    print(entry)

{'_id': ObjectId('6323f9d944646eea70730d54'), 'date': datetime.datetime(2021, 1, 31, 0, 0), 'Houston': '$1,385.37', 'San Antonio': '$1,244.32'}
{'_id': ObjectId('6323f9da44646eea70730d55'), 'date': datetime.datetime(2021, 2, 28, 0, 0), 'Houston': '$1,381.70', 'San Antonio': '$1,246.64'}
{'_id': ObjectId('6323f9da44646eea70730d56'), 'date': datetime.datetime(2021, 3, 31, 0, 0), 'Houston': '$1,387.38', 'San Antonio': '$1,256.56'}
{'_id': ObjectId('6323f9da44646eea70730d57'), 'date': datetime.datetime(2021, 4, 30, 0, 0), 'Houston': '$1,400.49', 'San Antonio': '$1,272.03'}
{'_id': ObjectId('6323f9da44646eea70730d58'), 'date': datetime.datetime(2021, 5, 31, 0, 0), 'Houston': '$1,427.69', 'San Antonio': '$1,293.88'}
{'_id': ObjectId('6323f9da44646eea70730d59'), 'date': datetime.datetime(2021, 6, 30, 0, 0), 'Houston': '$1,459.17', 'San Antonio': '$1,319.93'}
{'_id': ObjectId('6323f9da44646eea70730d5a'), 'date': datetime.datetime(2021, 7, 31, 0, 0), 'Houston': '$1,491.48', 'San Antonio': '$1,3

## WebScraping Data

### Houston and Surrounding Areas

In [15]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [16]:
url = "https://www.recenter.tamu.edu/data/housing-activity/#!/activity/MSA/Houston-The_Woodlands-Sugar_Land"
    
# Call visit on our browser and pass in the URL we want to scrape   
browser.visit(url)

In [17]:
table_data = browser.find_by_xpath('//*[@id="WebPartWPQ1"]/div[1]/div/div[2]/table').text
table_data[0]
table_data_split = table_data.split(" ")
table_data_split

['Date',
 'Sales',
 'Dollar\nVolume',
 'Average\nPrice',
 'Median\nPrice',
 'Total\nListings',
 'Months\nInventory\nJan',
 '1990',
 '2,500',
 '214,427,500',
 '85,771',
 '45,500',
 '22,318',
 '8.9\nFeb',
 '1990',
 '1,935',
 '171,518,400',
 '88,640',
 '66,231',
 '23,191',
 '10.5\nMar',
 '1990',
 '2,554',
 '219,963,250',
 '86,125',
 '67,052',
 '24,247',
 '10.4\nApr',
 '1990',
 '2,434',
 '221,267,638',
 '90,907',
 '66,949',
 '24,709',
 '10.5\nMay',
 '1990',
 '2,999',
 '258,048,955',
 '86,045',
 '65,923',
 '25,390',
 '10.2\nJun',
 '1990',
 '3,204',
 '295,988,724',
 '92,381',
 '70,439',
 '25,023',
 '9.6\nJul',
 '1990',
 '3,122',
 '285,072,942',
 '91,311',
 '69,618',
 '25,153',
 '9.4\nAug',
 '1990',
 '3,627',
 '337,151,412',
 '92,956',
 '69,310',
 '24,280',
 '8.7\nSep',
 '1990',
 '2,588',
 '217,407,528',
 '84,006',
 '64,794',
 '23,385',
 '8.4\nOct',
 '1990',
 '2,805',
 '236,966,400',
 '84,480',
 '65,307',
 '23,075',
 '8.3\nNov',
 '1990',
 '2,500',
 '214,427,500',
 '85,771',
 '45,500',
 '22,31

In [18]:
#Function to slice the raw data
def slice_per(source, step):
    return [source[i::step] for i in range(step)]

In [19]:
List_sliced_Data = slice_per(table_data_split, 7)

In [20]:
Table_df = pd.DataFrame({
    "Date":List_sliced_Data[0],
    "Sales":List_sliced_Data[1],
    "Dollar Volume":List_sliced_Data[2],
    "HOU Average Price":List_sliced_Data[3],
    "Median Price":List_sliced_Data[4],
    "Total Listings":List_sliced_Data[5],
    "Months Inventory":List_sliced_Data[6]
})

In [21]:
#View of the DataFrame
Table_df.head()

,Date,Sales,Dollar Volume,HOU Average Price,Median Price,Total Listings,Months Inventory
0,Date,Sales,Dollar\nVolume,Average\nPrice,Median\nPrice,Total\nListings,Months\nInventory\nJan
1,1990,"2,500","214,427,500","85,771","45,500","22,318",8.9\nFeb
2,1990,"1,935","171,518,400","88,640","66,231","23,191",10.5\nMar
3,1990,"2,554","219,963,250","86,125","67,052","24,247",10.4\nApr
4,1990,"2,434","221,267,638","90,907","66,949","24,709",10.5\nMay


In [22]:
Table_df_Clean = Table_df.drop([0])
Table_df_Clean

,Date,Sales,Dollar Volume,HOU Average Price,Median Price,Total Listings,Months Inventory
1,1990,"2,500","214,427,500","85,771","45,500","22,318",8.9\nFeb
2,1990,"1,935","171,518,400","88,640","66,231","23,191",10.5\nMar
3,1990,"2,554","219,963,250","86,125","67,052","24,247",10.4\nApr
4,1990,"2,434","221,267,638","90,907","66,949","24,709",10.5\nMay
5,1990,"2,999","258,048,955","86,045","65,923","25,390",10.2\nJun
...,...,...,...,...,...,...,...
387,2022,"10,085","4,088,374,167","405,392","330,000","10,839",1.1\nApr
388,2022,"9,337","3,942,527,921","422,248","340,870","12,182",1.3\nMay
389,2022,"9,988","4,340,285,197","434,550","350,000","13,692",1.5\nJun
390,2022,"10,123","4,362,282,525","430,928","351,330","17,689",1.9\nJul


In [23]:
Table_df_Clean2 = Table_df_Clean[['Date','HOU Average Price', 'Months Inventory']]
Table_df_Clean2

,Date,HOU Average Price,Months Inventory
1,1990,"85,771",8.9\nFeb
2,1990,"88,640",10.5\nMar
3,1990,"86,125",10.4\nApr
4,1990,"90,907",10.5\nMay
5,1990,"86,045",10.2\nJun
...,...,...,...
387,2022,"405,392",1.1\nApr
388,2022,"422,248",1.3\nMay
389,2022,"434,550",1.5\nJun
390,2022,"430,928",1.9\nJul


In [24]:
Table_df_Clean2 = Table_df_Clean2.astype({'Date':int},errors = 'raise')
Table_df_Clean2['Date'].dtypes

dtype('int32')

In [25]:
Table_df_Clean2021_only = Table_df_Clean2.loc[(Table_df_Clean2['Date'] == 2021),:]

In [26]:
Table_df_Clean2021_only

,Date,HOU Average Price,Months Inventory
373,2021,"322,682",1.8\nFeb
374,2021,"346,839",1.6\nMar
375,2021,"365,523",1.4\nApr
376,2021,"367,353",1.4\nMay
377,2021,"382,193",1.3\nJun
378,2021,"390,387",1.5\nJul
379,2021,"383,873",1.8\nAug
380,2021,"375,884",1.8\nSep
381,2021,"369,225",1.7\nOct
382,2021,"373,881",1.7\nNov


In [27]:
Table_df_Clean2021_only["Date"]= Table_df_Clean2021_only["Date"].astype(str)+"-"+Table_df_Clean2021_only["Months Inventory"].str[-3:]
Table_df_Clean2021_only.head()

C:\Users\HOBBYD~1\AppData\Local\Temp/ipykernel_75480/2738657502.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Table_df_Clean2021_only["Date"]= Table_df_Clean2021_only["Date"].astype(str)+"-"+Table_df_Clean2021_only["Months Inventory"].str[-3:]


,Date,HOU Average Price,Months Inventory
373,2021-Feb,"322,682",1.8\nFeb
374,2021-Mar,"346,839",1.6\nMar
375,2021-Apr,"365,523",1.4\nApr
376,2021-May,"367,353",1.4\nMay
377,2021-Jun,"382,193",1.3\nJun


In [28]:
#Final Clean Data for Houston
Table_df_Clean2021_only_Final = Table_df_Clean2021_only[['Date','HOU Average Price']]
Table_df_Clean2021_only_Final

,Date,HOU Average Price
373,2021-Feb,"322,682"
374,2021-Mar,"346,839"
375,2021-Apr,"365,523"
376,2021-May,"367,353"
377,2021-Jun,"382,193"
378,2021-Jul,"390,387"
379,2021-Aug,"383,873"
380,2021-Sep,"375,884"
381,2021-Oct,"369,225"
382,2021-Nov,"373,881"


### San Antonio and Surrounding Areas

In [29]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [30]:
SATX_url = "https://www.recenter.tamu.edu/data/housing-activity/#!/activity/MSA/San_Antonio-New_Braunfels"
    
# Call visit on our browser and pass in the URL we want to scrape   
browser.visit(SATX_url)

In [31]:
SATX_table_data = browser.find_by_xpath('//*[@id="WebPartWPQ1"]/div[1]/div/div[2]/table').text

In [32]:
SATX_table_data[0]
SATX_table_data_split = SATX_table_data.split(" ")

In [33]:
SATX_table_data_split

['Date',
 'Sales',
 'Dollar\nVolume',
 'Average\nPrice',
 'Median\nPrice',
 'Total\nListings',
 'Months\nInventory\nJan',
 '1990',
 '478',
 '35,062,256',
 '73,352',
 '63,830',
 '8,211',
 '17.2\nFeb',
 '1990',
 '363',
 '27,456,231',
 '75,637',
 '60,258',
 '9,280',
 '22.1\nMar',
 '1990',
 '553',
 '45,066,182',
 '81,494',
 '61,548',
 '9,335',
 '20.1\nApr',
 '1990',
 '429',
 '38,545,221',
 '89,849',
 '59,067',
 '9,545',
 '20.9\nMay',
 '1990',
 '537',
 '48,376,719',
 '90,087',
 '62,143',
 '9,588',
 '20.3\nJun',
 '1990',
 '682',
 '50,826,732',
 '74,526',
 '62,540',
 '9,221',
 '18.2\nJul',
 '1990',
 '638',
 '51,613,562',
 '80,899',
 '61,945',
 '9,087',
 '17.3\nAug',
 '1990',
 '776',
 '61,661,736',
 '79,461',
 '66,013',
 '8,751',
 '15.7\nSep',
 '1990',
 '490',
 '37,656,500',
 '76,850',
 '65,219',
 '8,731',
 '15.9\nOct',
 '1990',
 '468',
 '32,604,624',
 '69,668',
 '59,464',
 '8,580',
 '15.8\nNov',
 '1990',
 '478',
 '35,062,256',
 '73,352',
 '63,830',
 '8,211',
 '15.3\nDec',
 '1990',
 '430',
 '3

In [34]:
def slice_per(source, step):
    return [source[i::step] for i in range(step)]

In [35]:
List_sliced_Data = slice_per(SATX_table_data_split, 7)

In [36]:
SATX_Table_df = pd.DataFrame({
    "Date":List_sliced_Data[0],
    "Sales":List_sliced_Data[1],
    "Dollar Volume":List_sliced_Data[2],
    "SATX Average Price":List_sliced_Data[3],
    "Median Price":List_sliced_Data[4],
    "Total Listings":List_sliced_Data[5],
    "Months Inventory":List_sliced_Data[6]
})

In [37]:
SATX_Table_df.head()

,Date,Sales,Dollar Volume,SATX Average Price,Median Price,Total Listings,Months Inventory
0,Date,Sales,Dollar\nVolume,Average\nPrice,Median\nPrice,Total\nListings,Months\nInventory\nJan
1,1990,478,"35,062,256","73,352","63,830","8,211",17.2\nFeb
2,1990,363,"27,456,231","75,637","60,258","9,280",22.1\nMar
3,1990,553,"45,066,182","81,494","61,548","9,335",20.1\nApr
4,1990,429,"38,545,221","89,849","59,067","9,545",20.9\nMay


In [38]:
SATX_Table_df_Clean = SATX_Table_df.drop([0])
SATX_Table_df_Clean

,Date,Sales,Dollar Volume,SATX Average Price,Median Price,Total Listings,Months Inventory
1,1990,478,"35,062,256","73,352","63,830","8,211",17.2\nFeb
2,1990,363,"27,456,231","75,637","60,258","9,280",22.1\nMar
3,1990,553,"45,066,182","81,494","61,548","9,335",20.1\nApr
4,1990,429,"38,545,221","89,849","59,067","9,545",20.9\nMay
5,1990,537,"48,376,719","90,087","62,143","9,588",20.3\nJun
...,...,...,...,...,...,...,...
387,2022,"3,652","1,330,776,996","364,397","314,990","3,979",1.1\nApr
388,2022,"3,427","1,316,207,612","384,070","322,000","4,655",1.3\nMay
389,2022,"3,716","1,440,982,410","387,778","338,000","5,530",1.6\nJun
390,2022,"3,811","1,517,319,861","398,142","339,900","7,200",2.1\nJul


In [39]:
SATX_Table_df_Clean2 = SATX_Table_df_Clean[['Date','SATX Average Price', 'Months Inventory']]
SATX_Table_df_Clean2

,Date,SATX Average Price,Months Inventory
1,1990,"73,352",17.2\nFeb
2,1990,"75,637",22.1\nMar
3,1990,"81,494",20.1\nApr
4,1990,"89,849",20.9\nMay
5,1990,"90,087",20.3\nJun
...,...,...,...
387,2022,"364,397",1.1\nApr
388,2022,"384,070",1.3\nMay
389,2022,"387,778",1.6\nJun
390,2022,"398,142",2.1\nJul


In [40]:
SATX_Table_df_Clean2 = SATX_Table_df_Clean2.astype({'Date':int},errors = 'raise')
SATX_Table_df_Clean2['Date'].dtypes

dtype('int32')

In [41]:
SATX_Table_df_Clean2021_only = SATX_Table_df_Clean2.loc[(SATX_Table_df_Clean2['Date'] == 2021),:]

In [42]:
SATX_Table_df_Clean2021_only

,Date,SATX Average Price,Months Inventory
373,2021,"298,212",1.6\nFeb
374,2021,"309,799",1.4\nMar
375,2021,"312,571",1.2\nApr
376,2021,"323,832",1.2\nMay
377,2021,"334,964",1.2\nJun
378,2021,"339,670",1.3\nJul
379,2021,"344,579",1.6\nAug
380,2021,"343,165",1.7\nSep
381,2021,"342,042",1.7\nOct
382,2021,"352,445",1.7\nNov


In [43]:
SATX_Table_df_Clean2021_only["Date"]= SATX_Table_df_Clean2021_only["Date"].astype(str)+"-"+ SATX_Table_df_Clean2021_only["Months Inventory"].str[-3:]
SATX_Table_df_Clean2021_only.head()

C:\Users\HOBBYD~1\AppData\Local\Temp/ipykernel_75480/2735484600.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SATX_Table_df_Clean2021_only["Date"]= SATX_Table_df_Clean2021_only["Date"].astype(str)+"-"+ SATX_Table_df_Clean2021_only["Months Inventory"].str[-3:]


,Date,SATX Average Price,Months Inventory
373,2021-Feb,"298,212",1.6\nFeb
374,2021-Mar,"309,799",1.4\nMar
375,2021-Apr,"312,571",1.2\nApr
376,2021-May,"323,832",1.2\nMay
377,2021-Jun,"334,964",1.2\nJun


In [44]:
#Final Clean Data for San Antonio
SATX_Table_df_Clean2021_only_Final = SATX_Table_df_Clean2021_only[['Date','SATX Average Price']]
SATX_Table_df_Clean2021_only_Final

,Date,SATX Average Price
373,2021-Feb,"298,212"
374,2021-Mar,"309,799"
375,2021-Apr,"312,571"
376,2021-May,"323,832"
377,2021-Jun,"334,964"
378,2021-Jul,"339,670"
379,2021-Aug,"344,579"
380,2021-Sep,"343,165"
381,2021-Oct,"342,042"
382,2021-Nov,"352,445"


### Merging Houston and San Antonio Data

In [45]:
merged_cities = pd.merge(Table_df_Clean2021_only_Final, SATX_Table_df_Clean2021_only_Final, on="Date")

In [46]:
merged_cities

,Date,HOU Average Price,SATX Average Price
0,2021-Feb,"322,682","298,212"
1,2021-Mar,"346,839","309,799"
2,2021-Apr,"365,523","312,571"
3,2021-May,"367,353","323,832"
4,2021-Jun,"382,193","334,964"
5,2021-Jul,"390,387","339,670"
6,2021-Aug,"383,873","344,579"
7,2021-Sep,"375,884","343,165"
8,2021-Oct,"369,225","342,042"
9,2021-Nov,"373,881","352,445"


In [47]:
#Cleaning up the string values for calculation
merged_cities["HOU Average Price"] =merged_cities["HOU Average Price"].str.replace(',','')
merged_cities["SATX Average Price"] =merged_cities["SATX Average Price"].str.replace(',','')

#Calculating what monthly mortgage payment based on a 30yr loan

mthly_mortgage_HOU = merged_cities["HOU Average Price"].astype(int)/360
mthly_mortgage_SAT = merged_cities["SATX Average Price"].astype(int)/360

#Adding calculated monthly mortgage payment to the dataframe
merged_cities["Mthly Mortgage HOU (30yrs)"] = mthly_mortgage_HOU
merged_cities["Mthly Mortgage SAT (30yrs)"] = mthly_mortgage_SAT


In [48]:
merged_cities["Mthly Mortgage HOU (30yrs)"] = merged_cities["Mthly Mortgage HOU (30yrs)"].astype(float).map("${:,.2f}".format)
merged_cities["Mthly Mortgage SAT (30yrs)"] = merged_cities["Mthly Mortgage SAT (30yrs)"].astype(float).map("${:,.2f}".format)
merged_cities

,Date,HOU Average Price,SATX Average Price,Mthly Mortgage HOU (30yrs),Mthly Mortgage SAT (30yrs)
0,2021-Feb,322682,298212,$896.34,$828.37
1,2021-Mar,346839,309799,$963.44,$860.55
2,2021-Apr,365523,312571,"$1,015.34",$868.25
3,2021-May,367353,323832,"$1,020.42",$899.53
4,2021-Jun,382193,334964,"$1,061.65",$930.46
5,2021-Jul,390387,339670,"$1,084.41",$943.53
6,2021-Aug,383873,344579,"$1,066.31",$957.16
7,2021-Sep,375884,343165,"$1,044.12",$953.24
8,2021-Oct,369225,342042,"$1,025.62",$950.12
9,2021-Nov,373881,352445,"$1,038.56",$979.01


Based on the data above, it appears that the monthly cost to own is less than it is to rent a property in the associated area.

### Adding merged_cities dataframe to MongoDB

In [49]:
#Updating the MondoDB Collection with the data in the dataframe

owned_dict ={}

for row in merged_cities.index:
    #Creating the dictionary for the items in the dataframe
    owned_dict= {"date":merged_cities["Date"][row],
                  "Houston (owned)":merged_cities["Mthly Mortgage HOU (30yrs)"][row],
                  "San Antonio(owned)":merged_cities["Mthly Mortgage SAT (30yrs)"][row]}
    
    #Adding the dictionary to the database collection
    coll_Owned.insert_one(owned_dict)


In [50]:
# Display items in MongoDB collection
ownedTable = db.OwnedInfo.find()

for entry in ownedTable:
    print(entry)

{'_id': ObjectId('6323f9f944646eea70730d60'), 'date': '2021-Feb', 'Houston (owned)': '$896.34', 'San Antonio(owned)': '$828.37'}
{'_id': ObjectId('6323f9fa44646eea70730d61'), 'date': '2021-Mar', 'Houston (owned)': '$963.44', 'San Antonio(owned)': '$860.55'}
{'_id': ObjectId('6323f9fa44646eea70730d62'), 'date': '2021-Apr', 'Houston (owned)': '$1,015.34', 'San Antonio(owned)': '$868.25'}
{'_id': ObjectId('6323f9fa44646eea70730d63'), 'date': '2021-May', 'Houston (owned)': '$1,020.42', 'San Antonio(owned)': '$899.53'}
{'_id': ObjectId('6323f9fa44646eea70730d64'), 'date': '2021-Jun', 'Houston (owned)': '$1,061.65', 'San Antonio(owned)': '$930.46'}
{'_id': ObjectId('6323f9fa44646eea70730d65'), 'date': '2021-Jul', 'Houston (owned)': '$1,084.41', 'San Antonio(owned)': '$943.53'}
{'_id': ObjectId('6323f9fa44646eea70730d66'), 'date': '2021-Aug', 'Houston (owned)': '$1,066.31', 'San Antonio(owned)': '$957.16'}
{'_id': ObjectId('6323f9fa44646eea70730d67'), 'date': '2021-Sep', 'Houston (owned)': '$